In [1]:
# 1. 导入必要的库
import torch
import torch.optim as optim
from gesture_lstm import GestureLSTM  # 你的LSTM模型
from dataset import GestureDataset  # 导入我们定义的数据集类
from torch.utils.data import DataLoader
import numpy as np
import os
import copy


In [2]:
# 2. 加载数据
train_data_dir = "train_data"
val_data_dir = "val_data"

# 创建训练集和验证集的Dataset对象
train_dataset = GestureDataset(train_data_dir)
val_dataset = GestureDataset(val_data_dir)

# 创建DataLoader对象，用于批量加载数据
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 验证DataLoader是否正常工作
for data, labels in train_loader:
    print(data.shape, labels.shape)  # 应该输出 torch.Size([32, 30, 63]) 和 torch.Size([32])
    break  # 只打印一个batch

torch.Size([32, 30, 63]) torch.Size([32])


In [3]:
# 3. 定义LSTM模型
model = GestureLSTM(input_size=63, hidden_size=128, num_classes=8, num_layers=2, dropout=0.5)  # 8类（包括none）

# 4. 损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()  # 使用交叉熵损失
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [4]:
# 4. 训练过程
epochs = 50
early_stop_patience = 5  # 设置容忍度为5个epoch
best_val_acc = 0.0  # 最佳验证集准确率
patience_counter = 0  # 用于记录验证集准确率没有提升的次数
best_model_wts = copy.deepcopy(model.state_dict())  # 保存最佳模型的权重

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    # 训练循环
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        
        # 计算损失
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # 统计训练准确率
        _, preds = torch.max(outputs, dim=1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
        
        running_loss += loss.item()
    
    train_acc = correct_train / total_train
    train_loss = running_loss / len(train_loader)
    
    # 验证过程
    model.eval()
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
    
    val_acc = correct_val / total_val
    
    # 如果验证集准确率提高，保存模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())  # 保存最佳权重
        patience_counter = 0  # 重置容忍度计数
    else:
        patience_counter += 1  # 增加容忍度计数
    
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {train_loss:.4f}, Train Accuracy: {train_acc*100:.2f}%, Val Accuracy: {val_acc*100:.2f}%")
    
    # 如果连续多个epoch没有验证集准确率提升，提前停止训练
    if patience_counter >= early_stop_patience:
        print("Early stopping triggered")
        break

# 5. 保存最终模型（最佳模型）
model.load_state_dict(best_model_wts)  # 加载最佳模型权重
torch.save(model.state_dict(), 'final_model.pth')  # 保存模型权重
print("Model saved to 'final_model.pth'")

Epoch [1/50] - Loss: 0.0650, Train Accuracy: 98.42%, Val Accuracy: 98.91%
Epoch [2/50] - Loss: 0.0249, Train Accuracy: 99.24%, Val Accuracy: 99.61%
Epoch [3/50] - Loss: 0.0191, Train Accuracy: 99.41%, Val Accuracy: 99.38%
Epoch [4/50] - Loss: 0.0187, Train Accuracy: 99.40%, Val Accuracy: 99.77%
Epoch [5/50] - Loss: 0.0123, Train Accuracy: 99.64%, Val Accuracy: 99.84%
Epoch [6/50] - Loss: 0.0094, Train Accuracy: 99.72%, Val Accuracy: 95.44%
Epoch [7/50] - Loss: 0.0102, Train Accuracy: 99.67%, Val Accuracy: 99.47%
Epoch [8/50] - Loss: 0.0095, Train Accuracy: 99.70%, Val Accuracy: 99.83%
Epoch [9/50] - Loss: 0.0102, Train Accuracy: 99.70%, Val Accuracy: 74.01%
Epoch [10/50] - Loss: 0.0071, Train Accuracy: 99.76%, Val Accuracy: 99.86%
Epoch [11/50] - Loss: 0.0072, Train Accuracy: 99.77%, Val Accuracy: 99.87%
Epoch [12/50] - Loss: 0.0051, Train Accuracy: 99.82%, Val Accuracy: 99.69%
Epoch [13/50] - Loss: 0.0043, Train Accuracy: 99.88%, Val Accuracy: 99.95%
Epoch [14/50] - Loss: 0.0057, Trai